In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shapefile as shp 
import matplotlib as mpl

In [8]:
map_results_folder = '../map_results_onshore' 

In [6]:
def get_elcc_list(region,year,capacity,technology,add_on=None,scale_down=False):
    elcc_map_file = '%s/%s_%d_%d_MW_%s%s_results_map.csv' % (map_results_folder,region,year,capacity,technology,'' if add_on is None else '_'+add_on)
    elcc = pd.read_csv(elcc_map_file,index_col=0,header=0).values
    elcc = elcc[1::2,1::2] if scale_down else elcc
    elcc = elcc.flatten()
    elcc = elcc[np.logical_not(np.isnan(elcc))]
    return elcc

In [84]:
def get_scatter_size(x,y):

    if len(x) != len(y):
        raise RuntimeError('Arrays must be of same length.')

    uniq_x = np.unique(x)
    uniq_y = np.unique(y)

    weights = np.zeros((len(uniq_x),len(uniq_y)))

    # fill grid
    for i in range(len(x)):
        x_i = x[i]
        y_i = y[i]
        weights[x_i == uniq_x, y_i==uniq_y] += 1

    s = []
    
    #count
    for i in range(len(x)):
        s.append(max(float(weights[x[i] == uniq_x, y[i] == uniq_y]),3))
    
    return np.asarray(s)

In [86]:
def plot_diff_scatter(ax, elcc, elcc_w_storage, label_cbar=False, label_xaxis=False, label_yaxis=False):

    sns.set_theme(style="whitegrid")

    elcc_diff = elcc_w_storage - elcc

    s = get_scatter_size(elcc,elcc_diff)

    cmap = plt.get_cmap('plasma',50)

    n_min = int(np.min(elcc_w_storage)/10)*10
    n_max = np.max(elcc_w_storage)+10
    norm = mpl.colors.Normalize(vmin=n_min,vmax=n_max)

    ax.scatter(elcc, elcc_diff,s=s*2,c=elcc_w_storage,cmap=cmap,norm=norm,Linewidth=.5,edgecolors='#000000')
    ax.tick_params(axis='both',labelsize=15)
    ax.grid(False)

    # add colorbar    
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ticks=np.arange(n_min,n_max,10), boundaries=np.arange(n_min,n_max,2),ax=ax)
    cbar.ax.tick_params(labelsize=8)
    cbar.ax.tick_params(axis='y',length=2,width=1)
    cbar.outline.set_visible(False)

    # labels
    if label_xaxis: ax.set_xlabel('ELCC without Storage',fontsize=8)
    if label_yaxis: ax.set_ylabel('$\Delta$ ELCC',fontsize=8)
    if label_cbar: cbar.ax.set_ylabel('ELCC w/ Storage',fontsize=8)

    #label sizes
    ax.tick_params(axis='both',labelsize=8)

   
    return ax
    

In [87]:

fig, axs = plt.subplots(5,2,dpi=240, figsize=(7,10),constrained_layout=True)

year = 2019

i = 0

for region in ['basin','california','mountains','northwest','southwest']:
    for technology in ['solar','wind']:
        ax = axs[i//2,i%2]

        elcc = get_elcc_list(region,year,500,technology,scale_down=True)
        elcc_w_storage = get_elcc_list(region,year,500,technology,add_on='storage')
        
        ax = plot_diff_scatter(ax, elcc, elcc_w_storage,
                                 label_cbar=True if i%2==1 else False, 
                                 label_xaxis=True if i//2==4 else False,
                                 label_yaxis=True if i%2==0 else False)

        title = '%s %s' % (region.capitalize(),technology.capitalize())
        ax.set_title(title)

        i +=1

img_name = '../gallery/figure_4_storage_%d' % year
plt.savefig(img_name)  
plt.close()           
